In [2]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import plotly.graph_objects as go

     |████████████████████████████████| 32.7 MB 350 kB/s 


In [24]:
data = np.loadtxt('/content/drive/MyDrive/IMT2018050_Datathon3/datasets/bio-diseasome.mtx')

nodes = []
edges = []

#listing all nodes and edges
for edge in data[1:]:
    nodes.append(int(edge[0])) 
    nodes.append(int(edge[1]))
    edges.append((int(edge[0]),int(edge[1])))
#removing duplicate nodes
nodes = list(set(nodes))

#initializing graph and adding nodes and edges to it
graph = nx.Graph()
for node in nodes:
  graph.add_node(node)
for edge in edges:
  graph.add_edge(*edge)

In [ ]:
#degree distribution 
connectivity = sorted(set(dict(graph.degree()).values()))
hist = [list(dict(graph.degree()).values()).count(i)/float(nx.number_of_nodes(graph)) for i in connectivity]

plt.plot(hist)

In [29]:
def get_trace (graph, edges, nlayout):
  x_node = [nlayout[k+1][0] for k in range(516)]
  y_node = [nlayout[k+1][1] for k in range(516)]
  node_trace = go.Scatter(  x = x_node,
                            y = y_node,
                            mode = 'markers',
                            hoverinfo = 'text',
                            ids = nodes,
                            marker = dict(  showscale = True,
                                            colorscale = 'jet',
                                            reversescale = True,
                                            size = 10,
                                            colorbar = dict(
                                                thickness = 15,
                                                title = 'Degree of Node',
                                                xanchor = 'left',
                                                titleside = 'right'
                                            ),
                                            line_width = 2))
  
  x_edge = []
  y_edge = []
  for edge in edges:
      x_edge += [nlayout[edge[0]][0], nlayout[edge[1]][0], None]
      y_edge += [nlayout[edge[0]][1], nlayout[edge[1]][1], None]

  edge_trace = go.Scatter(  x = x_edge,
                            y = y_edge,
                            line = dict(width = 0.5, color = '#888'),
                            hoverinfo = 'none',
                            mode = 'lines')
  
  return node_trace, edge_trace


def get_node_link_diagram(graph, edges, nlayout, title):
  node_trace, edge_trace = get_trace (graph, edges, nlayout)
  node_adjacencies = []
  node_text = []
  for node, adjacencies in enumerate(graph.adjacency()):
      node_adjacencies.append(len(adjacencies[1]))
      node_text.append(node)
  node_trace.text = node_text    
  node_trace.marker.color = node_adjacencies
  node_trace.marker.size = node_adjacencies
  fig = go.Figure(  data = [edge_trace, node_trace],
                    layout = go.Layout(
                        title = "<br>"+title,
                        titlefont_size = 20,
                        width = 1000,
                        height = 500,
                        showlegend = False,
                        hovermode = 'closest',
                        margin = dict(b = 5, l = 5, r = 5, t = 5),
                        xaxis = dict(showgrid = False, zeroline = False, showticklabels = False),
                        yaxis = dict(showgrid = False, zeroline = False, showticklabels = False))
                        )
  fig.show()
  fig.write_html(title + ".html")


In [33]:
get_node_link_diagram(graph, edges, nx.kamada_kawai_layout(graph), "Kamada Kawai")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [86]:

def get_centrality_node_link_diagram(graph, edges, nlayout, title, measure):
  node_trace, edge_trace = get_trace (graph, edges, nlayout)
  node_adjacencies = []
  node_text = []
  for node, adjacencies in enumerate(graph.adjacency()):
      node_adjacencies.append(len(adjacencies[1]))
      node_text.append(node)
  measure_nodes = list(dict(sorted(measure.items(),key=lambda x:x[1],reverse=True)[0:5]))
  # print("Top 5 " + title + " nodes = " )
  # print(measure_nodes)
  node_trace.text = node_text
  node_trace.marker.color = node_adjacencies
  node_trace.marker.size = [50 if i in measure_nodes else 10 for i in graph.nodes]
  fig = go.Figure(  data = [edge_trace, node_trace],
                    layout = go.Layout(
                        title = "<br>"+title,
                        titlefont_size = 20,
                        width = 1000,
                        height = 500,
                        showlegend = False,
                        hovermode = 'closest',
                        margin = dict(b = 5, l = 5, r = 5, t = 5),
                        xaxis = dict(showgrid = False, zeroline = False, showticklabels = False),
                        yaxis = dict(showgrid = False, zeroline = False, showticklabels = False))
                        )
  fig.show()
  fig.write_html(title + ".html")


In [80]:
get_centrality_node_link_diagram(graph, edges, nx.kamada_kawai_layout(graph), "Betweenness", nx.betweenness_centrality(graph))
get_centrality_node_link_diagram(graph, edges, nx.kamada_kawai_layout(graph), "Closeness", nx.closeness_centrality(graph))
get_centrality_node_link_diagram(graph, edges, nx.kamada_kawai_layout(graph), "Eigen", nx.eigenvector_centrality(graph))
get_centrality_node_link_diagram(graph, edges, nx.kamada_kawai_layout(graph), "Degree", nx.degree_centrality(graph))

Top 5 Betweenness nodes = 
[80, 257, 121, 169, 113]
Top 5 Closeness nodes = 
[257, 121, 169, 80, 336]
Top 5 Eigen nodes = 
[93, 71, 457, 357, 186]
Top 5 Degree nodes = 
[93, 71, 163, 252, 457]
